# Imports

In [ ]:
import math
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import sklearn
import tensorflow as tf
import random
import shutil
import os
import time
import re

# Learning Models

In [ ]:
# FCN Model
def makeFCNModel(evalMetrics, learningRate, inputSize):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', input_shape = inputSize))
  model.add(tf.keras.layers.MaxPooling2D((3, 3)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer = tf.keras.optimizers.Adam(),
                loss = tf.keras.losses.BinaryCrossentropy(),
                metrics = evalMetrics)
  return model

# CNN Model
def makeCNNModel(evalMetrics, learningRate, inputSize):
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.Conv2D(64, (3, 3), activation = 'relu', input_shape = inputSize))
  model.add(tf.keras.layers.MaxPooling2D((3, 3)))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(128, activation = 'relu'))#Dense
  model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
  model.compile(optimizer = tf.keras.optimizers.Adam(),
                loss = tf.keras.losses.BinaryCrossentropy(),
                metrics = evalMetrics)
  return model

# Load Data

In [ ]:
df = pd.read_pickle('benchmark.pkl', compression='zip')
# Get ONLY completelly routed circuits
df2 = df.loc[df['FDRTotal'] == 0]
# Get all designs to train
circuitsToTrain = [x for x in df2.loc[df2['Design'].str.contains('80') == False].sort_values(by=['Design'])['Design']]

allPkls = ['/data/CSV/' + x[0:-3] + '/cts_' + x + '.pkl' for x in circuitsToTrain]
allPkls = [x for x in allPkls if os.path.exists(x)]

In [ ]:
# Get only designs within 75 and 85 except 80
allPkls = [x for x in allPkls if int(re.findall(r'\d+', x)[0]) < 86 and int(re.findall(r'\d+', x)[0]) > 74]

# Traning

In [ ]:
modelPath = 'fcn/'
model = 'fcn'

In [ ]:
modelPath = 'cnn/'
model = 'cnn'

In [ ]:
sizeBatch = 64  # almost 10% of chance to have viol \
                # is important to ensure that each batch has a decent chance of containing a few positive samples
numEpochs = 10
weights = {0: 0.5, 1: 50}
learningRate = 0.001
evalMetrics = [tf.keras.metrics.TruePositives(name='tp'),
               tf.keras.metrics.FalsePositives(name='fp'),
               tf.keras.metrics.TrueNegatives(name='tn'),
               tf.keras.metrics.FalseNegatives(name='fn'),
               tf.keras.metrics.BinaryAccuracy(name='accuracy'),
               tf.keras.metrics.Precision(name='precision'),
               tf.keras.metrics.Recall(name='recall'),
               tf.keras.metrics.AUC(name='auc')]


if os.path.exists(modelPath) == False:
  os.mkdir(modelPath)

models = [x for x in os.listdir(modelPath)]
lastRunEpoch = 0
inputSize = (22, 33, 33)
model = None
trainResultDF = pd.DataFrame()
if len(models) > 0:
  models.sort()
  lastModel = models[-1]
  lastRunEpoch = int(lastModel[lastModel.find('_')+1:lastModel.find('.')])
  model = pickle.load(open(modelPath+'model_'+str(lastRunEpoch)+'.pkl', 'rb'))
  trainResultDF = pickle.load(open('trainResultDF.pkl', 'rb'))
else:
  model = makeFCNModel(evalMetrics, learningRate, inputSize) if model == 'fcn' else makeCNNModel(evalMetrics, learningRate, inputSize)

for epoch in range(lastRunEpoch+1, numEpochs):
  random.shuffle(allPkls)
  for pkl in allPkls:
    trainDf = pd.read_pickle(pkl, compression='zip')
    trainDf = trainDf.reset_index(drop=True)
    valDf = trainDf.sample(frac=0.2)
    trainDf = trainDf.drop(valDf.index)

    labels = trainDf.pop(trainDf.columns.values[-1])
    valLabels = valDf.pop(valDf.columns.values[-1])
    trainHyperImages = np.array(trainDf).reshape(len(trainDf),22,33,33)
    valHyperImages = np.array(valDf).reshape(len(valDf),22,33,33)
    print('Epoch: ',epoch,' Training with:', pkl)
    train_history = model.fit(x=trainHyperImages,
                             y=labels,
                             verbose=2, #0 = silent, 1 = progress bar, 2 = one line per epoch
                             batch_size=sizeBatch,
                             validation_data=(valHyperImages, valLabels),
                             class_weight=weights)
    historyDf = pd.DataFrame(train_history.history)
    historyDf['epoch'] = epoch
    historyDf['design'] = pkl[pkl.rfind('/')+5:pkl.find('.')]
    trainResultDF = pd.concat([trainResultDF, historyDf])
  pickle.dump(model, open(modelPath+'model_'+str(epoch)+'.pkl', 'wb'))
  pickle.dump(trainResultDF, open('trainResultDF.pkl', 'wb'))

# Save Model

In [ ]:
pickle.dump(model, open('model.pkl', 'wb'))

# Load Model

In [ ]:
model = pickle.load(open('exp9Models/model_4.pkl', 'rb'))

In [ ]:
trainResultDF = pickle.load(open('EXP9trainResultDF.pkl', 'rb'))
trainResultDF.shape
# plt.plot(train_history.history['loss'][0:50])
# plt.plot(train_history.history['val_loss'][0:50])

In [ ]:
for epoch in range(min(trainResultDF['epoch']), max(trainResultDF['epoch'])+1):
  loss = sum(trainResultDF.loc[trainResultDF['epoch'] == epoch]['loss'])
  valLoss = sum(trainResultDF.loc[trainResultDF['epoch'] == epoch]['val_loss'])
  print('epoch:', epoch, ' loss: ', loss, ' valLoss: ', valLoss)

In [ ]:
trainResultDF.head

In [ ]:
design = 'jpeg_79'
sortedDF = trainResultDF.loc[trainResultDF['design'] == design].sort_values(by=['epoch'])
ytrain = [x for x in sortedDF['loss']]
yval = [x for x in sortedDF['val_loss']]
plt.plot(ytrain, label = "ytrain")
plt.plot(yval, label = "yval")
plt.legend()
plt.title(design)
plt.show()

# Evaluate

In [ ]:
def calculate_test_metrics(model, results):
  m = {}
  for name, value in zip(model.metrics_names, results):
      m[name] = value
  if m['precision'] + m['recall'] != 0:
      f_score = 2 * ((m['precision'] * m['recall'])/(m['precision'] + m['recall']))
      m['F-score'] = f_score
  sqrt = math.sqrt((m['tp']+m['fp'])*(m['tp']+m['fn'])*(m['tn']+m['fp'])*(m['tn']+m['fn']))
  if sqrt != 0:
      mcc = ((m['tp'] * m['tn']) - (m['fp'] * m['fn']))/sqrt
      m['MCC'] = mcc
  return m

In [ ]:
sizeBatch = 64 # is important to ensure that each batch has a decent chance of containing a few positive samples
# testPath = '/data/CSVWhole/jpeg/'
# testPkls = [testPath+x for x in  os.listdir(testPath)]
for circuit in os.listdir('/data/CSVWhole/'):
  i = 0
  resultDF = pd.DataFrame()
  for pkl in ['/data/CSVWhole/'+circuit+'/'+x for x in os.listdir('/data/CSVWhole/'+circuit)]:
    print(circuit, i, pkl)
    testDF = pd.read_pickle(pkl, compression='zip')
    testDF = testDF.sample(frac=1).reset_index(drop=True) #Shuffle all rows
    testLabels = testDF.pop(testDF.columns.values[-1])
    testHyperImages = np.array(testDF).reshape(len(testDF),22,33,33)
    baseline_results = model.evaluate(x=testHyperImages,
                                      y=testLabels,
                                      batch_size=sizeBatch)
    test_metrics = calculate_test_metrics(model, baseline_results)
    testDF = pd.DataFrame(test_metrics, index=[i])
    testDF['design'] = circuit
    resultDF = pd.concat([resultDF, testDF], ignore_index=True)
    i += 1
resultDF

# Benchmark Plot

In [ ]:
df = pd.read_pickle('benchmark.pkl', compression='zip')

In [ ]:
# Bar plot Routing Runtime Break Down
dfRuntime = df.loc[df['Design'].str.contains('80')].sort_values(by=['COMPONENTS'], ascending=False)
dfRuntime['Design'] = [x[0:x.rfind('_')] for x in dfRuntime['Design']]
dfRuntime = dfRuntime.set_index('Design')
dfRuntime = dfRuntime[['GR', 'IDR', 'FDR']]
ax = dfRuntime.plot.bar(stacked=True)
ax.set_ylabel('Runtime in seconds')
ax.set_xlabel('Designs (80% row utilization)')
ax.legend(["Global Routing", "Initial Detailed Routing", "Complete Detailed Routing"])
plt.title('Routing Runtime Break Down For 80% Row Utilization Designs')
# plt.savefig('routing_runtime.pdf')

In [ ]:
# Bar plot Routing Runtime Break Down
dfRuntime = df.loc[df['Design'].str.contains('80')].sort_values(by=['COMPONENTS'], ascending=False)
dfRuntime['Design'] = [x[0:x.rfind('_')] for x in dfRuntime['Design']]
dfRuntime = dfRuntime.set_index('Design')
dfRuntime = dfRuntime[['IDRShort']]
ax = dfRuntime.plot.bar(stacked=True)
ax.set_ylabel('Initial Detailed Routing Violations (IDRVs)')
ax.set_xlabel('Designs (80% row utilization)')
plt.title('Number of IDRVs For 80% Row Utilization Designs')

In [ ]:
# Plot avg IDR short (considering full routed circuits only)
idrShort = []
for x in range(70, 91):
  tempDf = df.loc[df['Design'].str.contains(str(x))]
  tempDf = tempDf.loc[tempDf['FDRTotal'] == 0]
  avgIDRShort = sum(tempDf['IDRShort']/len(tempDf))
  idrShort.append(avgIDRShort)

plt.plot([y for y in range(70, 91)], idrShort, color = 'r')
plt.xlabel("Design Density (Row Utilization %)")
plt.ylabel("Initial Detailed Routing Short Violations (IDRV)")
plt.title('Average IDR Short x Row Utilization (Only DRV Free Designs)')

In [ ]:
# Plot avg IDR short (considering full routed circuits only)
fdrRuntime = []
for x in range(70, 91):
  tempDf = df.loc[df['Design'].str.contains(str(x))]
  tempDf = tempDf.loc[tempDf['FDRTotal'] == 0]
  fdr = sum(tempDf['FDR']/len(tempDf))
  fdrRuntime.append(fdr)

plt.plot([y for y in range(70, 91)], fdrRuntime, color = 'r')
plt.xlabel("Design Density (Row Utilization %)")
plt.ylabel("Runtime (seconds)")
plt.title('Average Runtime to complete routing (Only DRV Free Designs)')

# Benchmark Info

# Compress All CSVs

# Backup

In [ ]:
# scaler = sklearn.preprocessing.StandardScaler()
# trainHyperImages = scaler.fit_transform(trainHyperImages)
# valHyperImages = scaler.transform(valHyperImages)

# scaler = sklearn.preprocessing.StandardScaler()
# labels = df[33*33*22]
# df[0:33*33*22-1] = scaler.fit_transform(df[0:33*33*22-1]).round(decimals=2)
# df[33*33*22] = labels

In [ ]:
# df = df.sample(frac=1).reset_index(drop=True) #Shuffle all rows
# dfVal = df.sample(frac=0.2)
# df = df.drop(dfVal.index)

# labels = df.pop(df.columns.values[-1])
# valLabels = dfVal.pop(dfVal.columns.values[-1])
# trainHyperImages = np.array(df).reshape(len(df),22,33,33)
# valHyperImages = np.array(dfVal).reshape(len(dfVal),22,33,33)